In [142]:
import pandas as pd
import os
import pandas as pd
from GrBinaryIPF import GrBinaryIPF
import math
import matplotlib.pyplot as plt
import itertools

import gurobipy as gp
from gurobipy import GRB

fpath =  "data/fantasy_football.pickle"

In [143]:
object = pd.read_pickle(fpath)

In [28]:
data = object[1]
num_of_player = 30
data = data[0:num_of_player]
data = data.transpose()
players = data.keys()
data

,Adam Thielen,Mike Evans,JuJu Smith-Schuster,Albert Wilson,Odell Beckham Jr.,Jarvis Landry,Courtland Sutton,Mohamed Sanu,Emmanuel Sanders,Tyler Boyd,...,Tyrell Williams,Dante Pettis,DeSean Jackson,Cooper Kupp,Alshon Jeffery,Julian Edelman,Josh Gordon,Donte Moncrief,Stefon Diggs,Brandin Cooks
Rank1,10,5,7,52,3,40,47,56,44,18,...,37,32,38,23,22,16,29,46,20,14
Rank2,9,4,8,52,2,34,40,56,46,19,...,37,32,39,18,25,13,23,48,29,12
Rank3,9,4,8,52,2,34,40,56,45,18,...,33,32,39,20,28,13,24,48,19,12
Rank4,9,7,6,52,5,32,43,55,42,16,...,37,33,39,19,24,13,23,46,21,12
Rank5,9,4,8,53,2,34,45,56,42,18,...,38,32,39,20,25,14,23,49,19,12
Rank6,8,1,6,52,2,32,43,57,42,21,...,39,31,38,18,22,16,24,50,19,13
Rank7,4,7,9,59,1,31,43,56,42,17,...,41,37,44,14,21,12,32,54,19,18
Rank8,9,4,6,52,5,35,45,55,50,20,...,40,30,38,18,21,16,24,48,22,13
Rank9,8,4,6,53,2,33,43,55,48,19,...,34,35,37,20,21,16,23,49,13,12
Rank10,12,4,8,55,2,34,40,56,49,21,...,39,35,32,27,20,15,17,50,19,13


In [29]:
itemList = data.keys()

In [30]:
G1 = []
G2 = []
row = data.iloc[25, :]
for i in range(0,num_of_player):
    if(row[i] == 0):
        G1.append(players[i])
    else:
        G2.append(players[i])

In [31]:
p1 = len(G1)/len(itemList)
p2 = len(G2)/len(itemList)
p = [p1,p2]

In [116]:

#Fair ILP


object = pd.read_pickle(r"data/top25_dfs.pickle")
data = object[1]
num_of_player = 35
data = data[0:num_of_player]
data = data.transpose()
players = data.keys()
nrow = len(data)
ncol = int(data.size/len(data))
l = ncol
rankDic = {}
rankList = []
for i in range(0,25):
    row = data.iloc[i, :]
    rank = {}
    for j in range(0,l):
        rankDic[(i,players[j])] = row[j]
        rank[players[j]] = row[j]
    rankList.append(rank)

itemList = data.keys()
itemList = itemList[0:l]
itemList

G1 = []
G2 = []
row = data.iloc[25, :]
group = row
for i in range(0,l):
    if(row[i] == 0):
        G1.append(players[i])
    else:
        G2.append(players[i])
        

p1 = len(G1)/len(itemList)
p2 = len(G2)/len(itemList)

d = 0.9999



G = []
G.append(G1)
G.append(G2)



p = []
p.append(p1)
p.append(p2)



combinations = [p for p in itertools.product(itemList, repeat=2)]
len(combinations)



scores = []
for tup in combinations:
    x,y = tup
    v = 0
    for i in range(0,25):
        if rankDic[(i,x)] < rankDic[(i,y)]:
            v = v +  1
    scores.append(v)




res = dict(zip(combinations, scores)) 

combinations, scores = gp.multidict(res)


# Declare and initialize model
m = gp.Model('RAP')

# Create decision variables for the RAP model
x = m.addVars(combinations,vtype=GRB.BINARY,name='x')


# Create  constraints
con = m.addConstrs((x[i,i] == 0  for i in itemList ), name='c0')


# Create  constraints
con = m.addConstrs((x[i,j] + x[j,i]  == 1   for j in itemList for i in itemList if i != j), name='c1')


# Create  constraints
#con = m.addConstrs((x[i,j] + x[j,k] + x[k,i]  >= 1 for j in itemList for i in itemList for k in itemList if (i!= j and j!= k and k != i)), name='c2')
#con = m.addConstrs( (gp.quicksum(p[u]*x[i, j] for j in itemList) - gp.quicksum(x[i, k] for k in G[u]) <= d  for i in itemList  for u in  range(0,len(G)) ) ,name='cfair2')



# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(scores), GRB.MINIMIZE)


# Save model for inspection
m.write('RAP.lp')


# Run optimization engine
m.optimize()


row = data.iloc[25, :]
len(m.getVars())


# Display optimal values of decision variables
i = 0
s = 0
scoredict = {}
for v in m.getVars():
   
    i = i + 1
    s = s + v.x
    #print(v.varName, v.x)
    if i % len(itemList) == 0:
        #print(i / len(itemList))
        g = 'g1'
        if (row[int(i / len(itemList)) - 1] == 1):
            g = 'g2'
        scoredict[itemList[int(i / len(itemList)) - 1]] = (s,g)
        s = 0 

#Display optimal total matching score
print('Total matching score: ', m.objVal)



RoutFairIlp = []

for w in sorted(scoredict, key=scoredict.get, reverse=False):
    print(w, scoredict[w])
    RoutFairIlp.append(w)
    

resRank = {}
for p in scoredict.keys():
    resRank[p] = scoredict[p][0]

print("output rank is = ",resRank)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1225 rows, 1225 columns and 2415 nonzeros
Model fingerprint: 0x261be0e9
Variable types: 0 continuous, 1225 integer (1225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7204.0000000
Presolve removed 1225 rows and 1225 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 617 7204 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.170000000000e+02, best bound 6.170000000000e+02, gap 0.0000%
Total matching score:  617.0
DeAndre Hopkins (0.0, 'g2')
Michael Thomas (1.0, 'g2')
Odell Beckham Jr. (2.0, 'g1')
Mike Evans (3.

In [38]:

def GrBinaryIPF(rank,group):
    Rho0 = []
    Rho1 = []
    for i in rank:
        if group[i] == 1:
            Rho0.append(i)
        else:
            Rho1.append(i)

    j = 1
    rankDic = {}
    for itm in rank:
        rankDic[itm] = j
        j = j + 1

    urgent = []
    Rout = []
    P1count = 0
    P0count = 0

    Fp0 = len(Rho0)/len(rank)
    Fp1 = len(Rho1)/len(rank)

    i = 1
    while len(Rho0) != 0 or len(Rho1) != 0:
        #print(Rout)
        if P1count >= len(Rho1):
            Rout.extend(Rho0[P0count:len(Rho0)])
            return Rout
        if P1count >= len(Rho0):
            Rout.extend(Rho1[P1count:len(Rho1)])
            return Rout

        if len(urgent) == 0:
            if rankDic[Rho1[P1count]] < rankDic[Rho0[P0count]]:
                Rout.append(Rho1[P1count])
                P1count = P1count + 1
            else:
                Rout.append(Rho0[P0count])
                P0count = P0count + 1
        else:
            if urgent[0] == 'P1':
                Rout.append(Rho1[P1count])
                P1count = P1count + 1
            else:
                Rout.append(Rho0[P0count])
                P0count = P0count + 1
            urgent = []
        # update urgent
        if Fp1 * (i + 1) - P1count >= 1:
            urgent.append('P1')

        if Fp0 * (i + 1) - P0count >= 1:
            urgent.append('P0')
        i = i + 1
        #print(i)
    return  Rout


In [123]:
Rout = []
rankList = []
for i in range(0,25):
    rank = list(resRank.values())
    rankList.append(rank)
    
rank = list(resRank.values())
ranktup = []
j = 0
for i in rank:
    ranktup.append((i,j))
    j = j + 1
ranktup.sort()
rank = []

for i,j in ranktup:
    rank.append(j)  

tup = []
for i in range(0,len(rank)):
    tup.append((rank[i],i))

tup.sort()
rank = []
for i,j in tup:
    rank.append(j)
group = row
rankList.append(rank)
Rout.append(GrBinaryIPF(rank,group))

In [124]:
players = [i for i in range(0,25)]

In [125]:
combinations = [p for p in itertools.product(players[0:l], repeat=2)]
combinations

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 21),
 (1, 22),
 (1, 23),
 (1, 24),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),
 (2, 19),
 (2, 20),
 (2, 21),
 (2, 22),
 (2, 23),
 (2, 24),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (3, 10),
 (3, 11),
 (3, 12),
 (3, 13),
 (3, 14),
 (3, 15),
 (3, 16),
 (3, 17),
 (3, 18),
 (3, 19),
 (3, 20),
 (3, 21),
 (3, 22),
 (3, 23),
 (3, 24),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4,

In [106]:
rankList[10][0]

7

In [55]:

#Fair ILP


object = pd.read_pickle(r"data/top25_dfs.pickle")
data = object[1]
num_of_player = 35
data = data[0:num_of_player]
data = data.transpose()
players = data.keys()
nrow = len(data)
ncol = int(data.size/len(data))
l = ncol
rankDic = {}
rankList = []
for i in range(0,25):
    row = data.iloc[i, :]
    rank = {}
    for j in range(0,l):
        rankDic[(i,players[j])] = row[j]
        rank[players[j]] = row[j]
    rankList.append(rank)

itemList = data.keys()
itemList = itemList[0:l]
itemList

G1 = []
G2 = []
row = data.iloc[25, :]
group = row
for i in range(0,l):
    if(row[i] == 0):
        G1.append(players[i])
    else:
        G2.append(players[i])
        

p1 = len(G1)/len(itemList)
p2 = len(G2)/len(itemList)

d = 0.9999



G = []
G.append(G1)
G.append(G2)



p = []
p.append(p1)
p.append(p2)



combinations = [p for p in itertools.product(itemList, repeat=2)]
len(combinations)



scores = []
for tup in combinations:
    x,y = tup
    v = 0
    for i in range(0,25):
        if rankDic[(i,x)] < rankDic[(i,y)]:
            v = v +  1
    scores.append(v)




res = dict(zip(combinations, scores)) 

combinations, scores = gp.multidict(res)


# Declare and initialize model
m = gp.Model('RAP')

# Create decision variables for the RAP model
x = m.addVars(combinations,vtype=GRB.BINARY,name='x')


# Create  constraints
con = m.addConstrs((x[i,i] == 0  for i in itemList ), name='c0')


# Create  constraints
con = m.addConstrs((x[i,j] + x[j,i]  == 1   for j in itemList for i in itemList if i != j), name='c1')


# Create  constraints
con = m.addConstrs((x[i,j] + x[j,k] + x[k,i]  >= 1 for j in itemList for i in itemList for k in itemList if (i!= j and j!= k and k != i)), name='c2')
#con = m.addConstrs( (gp.quicksum(p[u]*x[i, j] for j in itemList) - gp.quicksum(x[i, k] for k in G[u]) <= d  for i in itemList  for u in  range(0,len(G)) ) ,name='cfair2')



# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(scores), GRB.MINIMIZE)


# Save model for inspection
m.write('RAP.lp')


# Run optimization engine
m.optimize()


row = data.iloc[25, :]
len(m.getVars())


# Display optimal values of decision variables
i = 0
s = 0
scoredict = {}
for v in m.getVars():
   
    i = i + 1
    s = s + v.x
    #print(v.varName, v.x)
    if i % len(itemList) == 0:
        #print(i / len(itemList))
        g = 'g1'
        if (row[int(i / len(itemList)) - 1] == 1):
            g = 'g2'
        scoredict[itemList[int(i / len(itemList)) - 1]] = (s,g)
        s = 0 

#Display optimal total matching score
print('Total matching score: ', m.objVal)



RoutFairIlp = []

for w in sorted(scoredict, key=scoredict.get, reverse=False):
    print(w, scoredict[w])
    RoutFairIlp.append(w)
    

resRank = {}
for p in scoredict.keys():
    resRank[p] = scoredict[p][0]

print("output rank is = ",resRank)

def KendallTau(P,Q):
    combinations = [p for p in itertools.product(players[0:l], repeat=2)]
    distance = 0
    for tup in combinations:
        if P[tup[0]] < P[tup[1]] and  Q[tup[1]] < Q[tup[0]]:
            distance = distance + 1
    return distance




sumKTau = 0 
for i in range(0,len(rankList)):
    rankQ = rankList[i]
    ktau = KendallTau(resRank,rankQ)
    #print(ktau)
    sumKTau = sumKTau + ktau
avgKTau = sumKTau/len(rankList)
print(avgKTau)



Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 40495 rows, 1225 columns and 120225 nonzeros
Model fingerprint: 0xb18ed3e3
Variable types: 0 continuous, 1225 integer (1225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6339.0000000
Presolve removed 27405 rows and 630 columns
Presolve time: 0.15s
Presolved: 13090 rows, 595 columns, 39270 nonzeros
Variable types: 0 continuous, 595 integer (595 binary)
Found heuristic solution: objective 3759.0000000

Root relaxation: objective 6.170000e+02, 0 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     617.0000000 

In [133]:
def GrBinaryIPFDelta(rank,group):
    Rho0 = []
    Rho1 = []
    for i in rank:
        if group[i] == 1:
            Rho0.append(i)
        else:
            Rho1.append(i)

    j = 1
    rankDic = {}
    for itm in rank:
        rankDic[itm] = j
        j = j + 1

    urgent = []
    Rout = []
    P1count = 0
    P0count = 0

    Fp0 = len(Rho0)/len(rank)
    Fp1 = len(Rho1)/len(rank)

    i = 1
    while len(Rho0) != 0 or len(Rho1) != 0:
        if P1count >= len(Rho1):
            Rout.extend(Rho0[P0count:len(Rho0)])
            return Rout
        if P0count >= len(Rho0):
            Rout.extend(Rho1[P1count:len(Rho1)])
            return Rout

        if P1count < len(Rho1) and  P0count < len(Rho0):

            if len(urgent) == 0:
                if rankDic[Rho1[P1count]] < rankDic[Rho0[P0count]]:
                    Rout.append(Rho1[P1count])
                    P1count = P1count + 1
                else:
                    Rout.append(Rho0[P0count])
                    P0count = P0count + 1
            else:
                if urgent[0] == 'P1':
                    Rout.append(Rho1[P1count])
                    P1count = P1count + 1
                else:
                    Rout.append(Rho0[P0count])
                    P0count = P0count + 1
                urgent = []
        else:
            break
        # update urgent
        delta = 0

        if Fp1 * (i + 1) - P1count >= delta:
            urgent.append('P1')

        if Fp0 * (i + 1) - P0count >= delta:
            urgent.append('P0')
        i = i + 1
        #print(i)

    return  Rout

import timeit


def kendalTau(P,Q):
    qInv = {}
    pInv = {}
    for key in P:
        #print(key, P[key])
        val = P[key]
        pInv[val] = key
    for key in Q:
        #print(key, Q[key])
        val = Q[key]
        qInv[val] = key

    qTrans = {}
    #qTransInv = {}
    for key in Q:
        #print(key, Q[key])
        value = Q[key]
        newVal = pInv[value]
        qTrans[key] = newVal
        #qTransInv[newVal] = key

    dis = 0
    for key in qTrans:
        dis = dis + abs(key - qTrans[key])

    return dis









import pandas as pd

object = pd.read_pickle(r'data/top25_dfs.pickle')

data = object[1]
num_of_player = 50
#data = data[0:num_of_player]
data = data.transpose()
players = data.keys()

itemList = data.keys()
G1 = []
G2 = []
row = data.iloc[25, :num_of_player]
for i in range(0,num_of_player):
    if(row[i] == 0):
        G1.append(players[i])
    else:
        G2.append(players[i])

p1 = len(G1)/len(itemList)
p2 = len(G2)/len(itemList)


groupInfo = data.iloc[25, :]

playeridDic = {}
j = 0
for p in players :
    playeridDic[p] = j
    j = j + 1

# group = {}
# j = 0
# for i in groupInfo:
#     group[j] = i
#     j = j + 1

inputRankList = []

#start = timer()



result = []
for rankIds in range(0,1):

    rankinfo = data.iloc[rankIds, :num_of_player]
    ranktup = []
    j = 0
    for i in rankinfo:
        ranktup.append((i, j))
        j = j + 1
    ranktup.sort()
    rank = []

    for i, j in ranktup:
        rank.append(j)


    group = {}
    for i in range(0,len(rank)):
        group[i] = groupInfo[i]

    rank = list(resRank.values())
    rout = GrBinaryIPFDelta(rank, group)
    result.append((rank,rout))
    #print(rout)

print(rout)

import itertools

items = []
for i in range(0,len(rank)):
    items.append(i)
combinations = [p for p in itertools.product(items, repeat=2)]
print(len(combinations))

def KendallTau(P,Q,combinations):
    distance = 0
    for tup in combinations:
        if int(P[tup[0]]) < int(P[tup[1]]) and  int(Q[tup[1]]) < int(Q[tup[0]]):
            distance = distance + 1
    return distance


#rand rapf result
pick =  0 #random.randint(0,len(result) - 1)
#print(pick)
rankpicked,fairRankPicked =  result[pick]

distance  = 0
for rank, fairRank in result:

    P = {}
    Q = {}
    for i in range(0,len(rank)):
        P[rank[i]] = i
        Q[fairRankPicked[i]] = i

    distance = distance  + KendallTau(P,Q,combinations)

print(distance)

[7.0, 32.0, 3.0, 21.0, 5.0, 34.0, 2.0, 12.0, 27.0, 8.0, 29.0, 11.0, 1.0, 28.0, 17.0, 6.0, 31.0, 4.0, 33.0, 20.0, 0.0, 25.0, 24.0, 14.0, 15.0, 10.0, 16.0, 13.0, 30.0, 22.0, 9.0, 19.0, 26.0, 23.0, 18.0]
1225
19


In [ ]:
# Approximate Multi valued IPF

In [136]:
import pandas as pd
import random
import math

import networkx as nx
import networkx as nx
from networkx.algorithms import bipartite
import random
import math


df = pd.read_csv('data/movieLens_data.csv')

allGenre = []
group = []
genre = df['genre']
result = []
numberOfItem = 200#len(df.iloc[:, 1])
delta = 0

allGenre = []
allgroup = []
for g in genre:
    if g not in allGenre:
        allGenre.append(g)
    allgroup.append(allGenre.index(g))

for colnum in range(1,2):


    movieIds = df.iloc[:, colnum]
    movieDic = {}
    rank = []

    movieIdarray = []
    for i in movieIds:
        if i <= numberOfItem:
            movieIdarray.append(i)

    for i in range(1,len(movieIdarray)+1):
        movieId = movieIdarray[i-1]
        groupId = allgroup[movieId-1]
        movieDic[i] = movieId
        rank.append(i)
        group.append(groupId)



    rank = rank[0:numberOfItem]
    group = group[0:numberOfItem]




    rankGrp = {}
    for i in range(0, len(rank)):
        rankGrp[rank[i]] = group[i]

    grpCount = {}
    for i in group:
        grpCount[i] = 0

    rankGrpPos = {}
    for i in rank:
        grpCount[rankGrp[i]] = grpCount[rankGrp[i]] + 1
        rankGrpPos[i] = grpCount[rankGrp[i]]

    rankRange = {}
    for item in rank:
        i = rankGrpPos[item]
        n = numberOfItem
        fp = grpCount[rankGrp[item]]
        r1 = math.floor((i-1 - delta) * n / fp)+1
        r2 = math.ceil((i + delta) * n / fp)
        if r2 > numberOfItem:
            r2 = numberOfItem
        rankRange[item] = (r1, r2)

    B = nx.Graph()
    top_nodes = []
    bottom_nodes = []

    for i in rank:
        top_nodes.append(i)
        bottom_nodes.append(str(i))
    B.add_nodes_from(top_nodes, bipartite=0)
    B.add_nodes_from(bottom_nodes, bipartite=1)

    for i in rank:
        r1, r2 = rankRange[i]
        # print(r1,r2)
        for j in range(1, numberOfItem+1):
            if j >= r1 and j <= r2:
                #print(i,j)
                B.add_edge(i, str(j), weight=abs(i - j))
            # else:
            #     B.add_edge(i, str(j), weight=100000000000)
            #     # print(i,j)

    my_matching = nx.algorithms.bipartite.minimum_weight_full_matching(B, top_nodes, "weight")

    #print(my_matching)

    rank1 = []
    rank2 = []
    for i in range(0,numberOfItem):
        rank2.append(0)

    for i in range(1,numberOfItem+1):
        #print(my_matching[i])
        rank1.append(i)
        rank2[int(my_matching[i]) -1 ] = i


    inputRank = []
    fairRank = []

    for i in range(0,numberOfItem):
        rn1 = rank1[i]
        rn2 = rank2[i]
        inputRank.append(movieDic[rn1])
        fairRank.append(movieDic[rn2])



    result.append((inputRank,fairRank))

#print(result)


import itertools

items = []
for i in result[0][0]:
    items.append(i)
combinations = [p for p in itertools.product(items, repeat=2)]
print(len(combinations))

def KendallTau(P,Q,combinations):
    distance = 0
    for tup in combinations:
        if int(P[tup[0]]) < int(P[tup[1]]) and  int(Q[tup[1]]) < int(Q[tup[0]]):
            distance = distance + 1
    return distance


#rand rapf result


minAvg = 10000000
fairRankOutput = []
for rankPicked,fairRankPicked in result:
    distance = 0
    for rank, fairRank in result:
        P = {}
        Q = {}
        for i in range(0,len(rank)):
            P[rank[i]] = i
            Q[fairRankPicked[i]] = i

        distance = distance + KendallTau(P,Q,combinations)
    avgDistance = distance/len(result)
    print(len(result))
    if avgDistance < minAvg:
        minAvg = avgDistance
        fairRankOutput = Q

print(minAvg)
#print(Q)




40000
1
2756.0


In [137]:
#AlgRAPF

In [138]:
import pandas as pd
import random
import math
from timeit import default_timer as timer

def GrBinaryIPFDelta(rank,group):
    Rho0 = []
    Rho1 = []
    for i in rank:
        if group[i] == 1:
            Rho0.append(i)
        else:
            Rho1.append(i)

    j = 1
    rankDic = {}
    for itm in rank:
        rankDic[itm] = j
        j = j + 1

    urgent = []
    Rout = []
    P1count = 0
    P0count = 0

    Fp0 = len(Rho0)/len(rank)
    Fp1 = len(Rho1)/len(rank)

    i = 1
    while len(Rho0) != 0 or len(Rho1) != 0:
        if P1count >= len(Rho1):
            Rout.extend(Rho0[P0count:len(Rho0)])
            return Rout
        if P0count >= len(Rho0):
            Rout.extend(Rho1[P1count:len(Rho1)])
            return Rout

        if P1count < len(Rho1) and  P0count < len(Rho0):

            if len(urgent) == 0:
                if rankDic[Rho1[P1count]] < rankDic[Rho0[P0count]]:
                    Rout.append(Rho1[P1count])
                    P1count = P1count + 1
                else:
                    Rout.append(Rho0[P0count])
                    P0count = P0count + 1
            else:
                if urgent[0] == 'P1':
                    Rout.append(Rho1[P1count])
                    P1count = P1count + 1
                else:
                    Rout.append(Rho0[P0count])
                    P0count = P0count + 1
                urgent = []
        else:
            break
        # update urgent
        delta = 0

        if Fp1 * (i + 1) - P1count >= delta:
            urgent.append('P1')

        if Fp0 * (i + 1) - P0count >= delta:
            urgent.append('P0')
        i = i + 1
        #print(i)

    return  Rout

import timeit


def kendalTau(P,Q):
    qInv = {}
    pInv = {}
    for key in P:
        #print(key, P[key])
        val = P[key]
        pInv[val] = key
    for key in Q:
        #print(key, Q[key])
        val = Q[key]
        qInv[val] = key

    qTrans = {}
    #qTransInv = {}
    for key in Q:
        #print(key, Q[key])
        value = Q[key]
        newVal = pInv[value]
        qTrans[key] = newVal
        #qTransInv[newVal] = key

    dis = 0
    for key in qTrans:
        dis = dis + abs(key - qTrans[key])

    return dis






import pandas as pd

object = pd.read_pickle(r'data/top25_dfs.pickle')

data = object[1]
num_of_player = 30
#data = data[0:num_of_player]
data = data.transpose()
players = data.keys()

itemList = data.keys()
G1 = []
G2 = []
row = data.iloc[25, :num_of_player]
for i in range(0,num_of_player):
    if(row[i] == 0):
        G1.append(players[i])
    else:
        G2.append(players[i])

p1 = len(G1)/len(itemList)
p2 = len(G2)/len(itemList)


groupInfo = data.iloc[25, :]

playeridDic = {}
j = 0
for p in players :
    playeridDic[p] = j
    j = j + 1

# group = {}
# j = 0
# for i in groupInfo:
#     group[j] = i
#     j = j + 1

inputRankList = []

start = timer()



result = []
for rankIds in range(0,25):

    rankinfo = data.iloc[rankIds, :num_of_player]
    ranktup = []
    j = 0
    for i in rankinfo:
        ranktup.append((i, j))
        j = j + 1
    ranktup.sort()
    rank = []

    for i, j in ranktup:
        rank.append(j)


    group = {}
    for i in range(0,len(rank)):
        group[i] = groupInfo[i]


    rout = GrBinaryIPFDelta(rank, group)
    result.append((rank,rout))
    #print(rout)



import itertools

items = []
for i in range(0,len(rank)):
    items.append(i)
combinations = [p for p in itertools.product(items, repeat=2)]
print(len(combinations))

def KendallTau(P,Q,combinations):
    distance = 0
    for tup in combinations:
        if int(P[tup[0]]) < int(P[tup[1]]) and  int(Q[tup[1]]) < int(Q[tup[0]]):
            distance = distance + 1
    return distance


#rand rapf result


minAvg = 10000000

for rankPicked,fairRankPicked in result:
    distance = 0
    for rank, fairRank in result:
        P = {}
        Q = {}
        for i in range(0,len(rank)):
            P[rank[i]] = i
            Q[fairRankPicked[i]] = i

        distance = distance + KendallTau(P,Q,combinations)
    avgDistance = distance/len(result)
    #print(avgDistance)
    if avgDistance < minAvg:
        minAvg = avgDistance

print(minAvg)






900
44.92


In [139]:
# randAlgRAPF

In [140]:
import pandas as pd
import random
import math
from timeit import default_timer as timer

def GrBinaryIPFDelta(rank,group):
    Rho0 = []
    Rho1 = []
    for i in rank:
        if group[i] == 1:
            Rho0.append(i)
        else:
            Rho1.append(i)

    j = 1
    rankDic = {}
    for itm in rank:
        rankDic[itm] = j
        j = j + 1

    urgent = []
    Rout = []
    P1count = 0
    P0count = 0

    Fp0 = len(Rho0)/len(rank)
    Fp1 = len(Rho1)/len(rank)

    i = 1
    while len(Rho0) != 0 or len(Rho1) != 0:
        if P1count >= len(Rho1):
            Rout.extend(Rho0[P0count:len(Rho0)])
            return Rout
        if P0count >= len(Rho0):
            Rout.extend(Rho1[P1count:len(Rho1)])
            return Rout

        if P1count < len(Rho1) and  P0count < len(Rho0):

            if len(urgent) == 0:
                if rankDic[Rho1[P1count]] < rankDic[Rho0[P0count]]:
                    Rout.append(Rho1[P1count])
                    P1count = P1count + 1
                else:
                    Rout.append(Rho0[P0count])
                    P0count = P0count + 1
            else:
                if urgent[0] == 'P1':
                    Rout.append(Rho1[P1count])
                    P1count = P1count + 1
                else:
                    Rout.append(Rho0[P0count])
                    P0count = P0count + 1
                urgent = []
        else:
            break
        # update urgent
        delta = 0

        if Fp1 * (i + 1) - P1count >= delta:
            urgent.append('P1')

        if Fp0 * (i + 1) - P0count >= delta:
            urgent.append('P0')
        i = i + 1
        #print(i)

    return  Rout

import timeit


def kendalTau(P,Q):
    qInv = {}
    pInv = {}
    for key in P:
        #print(key, P[key])
        val = P[key]
        pInv[val] = key
    for key in Q:
        #print(key, Q[key])
        val = Q[key]
        qInv[val] = key

    qTrans = {}
    #qTransInv = {}
    for key in Q:
        #print(key, Q[key])
        value = Q[key]
        newVal = pInv[value]
        qTrans[key] = newVal
        #qTransInv[newVal] = key

    dis = 0
    for key in qTrans:
        dis = dis + abs(key - qTrans[key])

    return dis









import pandas as pd

object = pd.read_pickle(r'data/top25_dfs.pickle')

data = object[1]
num_of_player = 50
#data = data[0:num_of_player]
data = data.transpose()
players = data.keys()

itemList = data.keys()
G1 = []
G2 = []
row = data.iloc[25, :num_of_player]
for i in range(0,num_of_player):
    if(row[i] == 0):
        G1.append(players[i])
    else:
        G2.append(players[i])

p1 = len(G1)/len(itemList)
p2 = len(G2)/len(itemList)


groupInfo = data.iloc[25, :]

playeridDic = {}
j = 0
for p in players :
    playeridDic[p] = j
    j = j + 1

# group = {}
# j = 0
# for i in groupInfo:
#     group[j] = i
#     j = j + 1

inputRankList = []

start = timer()



result = []
for rankIds in range(0,1):

    rankinfo = data.iloc[rankIds, :num_of_player]
    ranktup = []
    j = 0
    for i in rankinfo:
        ranktup.append((i, j))
        j = j + 1
    ranktup.sort()
    rank = []

    for i, j in ranktup:
        rank.append(j)


    group = {}
    for i in range(0,len(rank)):
        group[i] = groupInfo[i]


    rout = GrBinaryIPFDelta(rank, group)
    result.append((rank,rout))
    #print(rout)



import itertools

items = []
for i in range(0,len(rank)):
    items.append(i)
combinations = [p for p in itertools.product(items, repeat=2)]
print(len(combinations))

def KendallTau(P,Q,combinations):
    distance = 0
    for tup in combinations:
        if int(P[tup[0]]) < int(P[tup[1]]) and  int(Q[tup[1]]) < int(Q[tup[0]]):
            distance = distance + 1
    return distance


#rand rapf result
pick =  0 #random.randint(0,len(result) - 1)
#print(pick)
rankpicked,fairRankPicked =  result[pick]

distance  = 0
for rank, fairRank in result:

    P = {}
    Q = {}
    for i in range(0,len(rank)):
        P[rank[i]] = i
        Q[fairRankPicked[i]] = i

    distance = distance  + KendallTau(P,Q,combinations)

print(distance/len(result))

2500
79.0
